In [1]:
!pwd

/data5/galaxy/shell_dir/2018_3_17/jupyter_shell


In [2]:
import os
import shutil
import glob
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
from itertools import permutations
from multiprocessing import Pool

In [25]:
m6a_dir = "/data5/galaxy/project/methyl_m6a/data/diff_m6a_peak/macs2_bdgdiff/combination/"
expre_dir = "/data5/galaxy/project/expression/DESeq2/"
expre_file = "/data5/galaxy/project/expression/stringtie/Total-RPKM.txt"
subdir_list = ["heart_vs_brain", "kid_vs_brain", "lung_vs_brain", "heart_vs_kid", "lung_vs_heart", "lung_vs_kid"]
#
df_total_exp = get_expre()
#
for sub_dir in subdir_list:
    print(sub_dir)
    df_gene_up, df_gene_down = class_gene_by_diffexpre(os.path.join(expre_dir, "%s.csv" % sub_dir))
    df_methyl_up, df_methyl_down = class_methyl_run(sub_dir)
    tissue_1, tissue_2 = sub_dir.split("_")[0], sub_dir.split("_")[-1]
    # term_dict = dict(zip(["DNA methyl up", "DNA methyl down"], [df_methyl_up, df_methyl_down]))
    for tissue in [tissue_1, tissue_2]:
        print(tissue)
        #for term, df_m in term_dict.items():
            # print(term)
        df_m6a = get_m6a(tissue)
        df_expre = df_total_exp[["name", tissue]]
        print("#### coorelation between expression && m6a!")
        print("expression vs m6a without methylation")
        df = df_m6a.merge(df_expre, on="name", how="left").dropna()
#         print(stats.spearmanr(df[tissue], df["m6a_score"]))

        ##
        ##
        print("expression vs m6a with methylation")
        df_total_methyl = get_methylation(tissue)
        # df_methyl = df_total_methyl.merge(df_m, on="name", how="right").dropna()
        df = df_m6a.merge(df_expre, on="name", how="left").merge(df_methyl, on="name", how="left").dropna()
        df_map_1 = df[(df["name"].isin(df_gene_down.name)) & (df["name"].isin(df_methyl_up.name))]
        df_map_2 = df[(df["name"].isin(df_gene_up.name)) & (df["name"].isin(df_methyl_down.name))]
        df_coor = pd.concat([df_map_1, df_map_2])
#         print(df_coor.head())
        print(stats.spearmanr(df_coor[tissue], df_coor["m6a_score"]))
#             print("#### coorelation between expression && methylation!")
#             print(stats.spearmanr(df[tissue], df["methyl"]))
#             print(stats.pearsonr(df[tissue], df["methyl"]))
#             #
#             print("#### coorelation between methylation && m6a!")
#             print(stats.spearmanr(df["methyl"], df["m6a_score"]))
#             print(stats.pearsonr(df["methyl"], df["m6a_score"]))
#     break

heart_vs_brain
heart
#### coorelation between expression && m6a!
expression vs m6a without methylation
SpearmanrResult(correlation=-0.2249027381571596, pvalue=1.02332899971635e-179)
expression vs m6a with methylation
SpearmanrResult(correlation=-0.061834730247670854, pvalue=0.32249826407626503)
brain
#### coorelation between expression && m6a!
expression vs m6a without methylation
SpearmanrResult(correlation=-0.2278449261923644, pvalue=4.573013679409501e-199)
expression vs m6a with methylation
SpearmanrResult(correlation=-0.0741689793823157, pvalue=0.2176613463145204)
kid_vs_brain
kid
#### coorelation between expression && m6a!
expression vs m6a without methylation
SpearmanrResult(correlation=-0.2548717810973115, pvalue=2.0159091753688507e-289)
expression vs m6a with methylation
SpearmanrResult(correlation=-0.01684814055294072, pvalue=0.7618486085643068)
brain
#### coorelation between expression && m6a!
expression vs m6a without methylation
SpearmanrResult(correlation=-0.22784492619236

In [ ]:
def 

In [3]:
def class_methyl_run(sub_dir):
    df_methyl = foldchange_of_methylation(sub_dir)
    df_up, df_down = class_methyl(df_methyl)
    return df_up.drop_duplicates(), df_down.drop_duplicates()
# 81--brain; 83--heart; 86--kid; 88--lung;
# ["heart_vs_brain", "kid_vs_brain", "lung_vs_brain", "heart_vs_kid", "lung_vs_heart", "lung_vs_kid"]
def foldchange_of_methylation(sub_dir):
    os.chdir("/data5/galaxy/project/methyl_m6a/data/roimethstat/")
    map_dict = {"brain": "E081", "heart": "E083", "kid": "E086", "lung": "E088"}
    tissue_1, tissue_2 = sub_dir.split("_")[0], sub_dir.split("_")[-1]
    try:
        num_1, num_2 = map_dict[tissue_1], map_dict[tissue_2]
    except KeyError:
        print("%s %s didn't have acoording DNA methylation data!" % (tissue_1, tissue_2))
    df_1 = pd.read_table("promoter_%s.bed" % num_1, header=None, names=["chr", "s", "e", "n", tissue_1, "strand"])
    df_1["name"] = df_1["n"].str.split(":").str[0]
    df_2 = pd.read_table("promoter_%s.bed" % num_2, header=None, names=["chr", "s", "e", "n", tissue_2, "strand"])
    df_2["name"] = df_2["n"].str.split(":").str[0]
    df_methyl = df_1.merge(df_2, on="name").dropna()
    df_methyl["logFC"] = np.log2(df_methyl[tissue_1].astype(float) + 0.01) - np.log2(df_methyl[tissue_2].astype(float) + 0.01)
    df_methyl = df_methyl[["name", "logFC"]]
#     print(df_1[tissue_1].mean(), df_2[tissue_2].mean())
    return df_methyl

def class_methyl(df):
    df_up = df[(df["logFC"] > 0) & (abs(df["logFC"]) > 0.585)]
    df_down = df[(df["logFC"] < 0) & (abs(df["logFC"]) > 0.585)]
    return df_up, df_down

In [13]:
def get_m6a(tissue_name):
    m6a_dir = "/data5/xlj/new_tissue_m6A/Macs2/Filter/Merge-Dup/peak2gene/gene_m6A_level"
    os.chdir(m6a_dir)
    tissue_list = ["brain", "heart", "kid", "lung"]
    real_list = ["Brain", "Heart", "Kidney", "Lung"]
    map_dict = dict(zip(tissue_list, real_list))
    try:
        m6a_file = map_dict[tissue_name]
    except KeyError:
        print("%s doesn't have the according" % m6a_dir)
    df = pd.read_table(m6a_file, sep="\t", header=None, names=["name", "m6a_score"])
    df["name"] = df["name"].str.split(".").str[0]
    return df

In [11]:
def get_expre():
    df = pd.read_table(expre_file, sep="\t")
    df.columns = ["gene", "brain", "heart", "kid", "liver", "pla", "lung", "mus", "sto"]
    df["name"] = df["gene"].str.split(".").str[0]
    return df

In [18]:
def get_methylation(tissue):
    os.chdir("/data5/galaxy/project/methyl_m6a/data/roimethstat/")
    map_dict = {"brain": "E081", "heart": "E083", "kid": "E086", "lung": "E088"}
    try:
        num = map_dict[tissue]
    except KeyError:
        print("%s didn't have acoording DNA methylation data!" % tissue)
    df = pd.read_table("promoter_%s.bed" % num, header=None, names=["chr", "s", "e", "n", "methyl", "strand"])
    df["name"] = df["n"].str.split(":").str[0]
    df["methyl"] = df["methyl"].astype(float)
    df = df[["name", "methyl"]]
    return df

In [21]:
def format_gene_name(df):
    df["name"] = df["Unnamed: 0"].str.split(".").str[0]
#     df = df.set_index(["name"])
    del df["Unnamed: 0"]
    return df

def class_gene_by_diffexpre(DESeq2_file):
    df = pd.read_csv(DESeq2_file)
    df = df[(abs(df.log2FoldChange) > 1) & (df.padj < 0.05)]
    df_up, df_down = df.copy()[df.log2FoldChange > 0], df.copy()[df.log2FoldChange < 0]
#     print(len(df_up), len(df_down))
    return format_gene_name(df_up), format_gene_name(df_down)